# FPI Model

### This model compares ESPN FPI (Football Power Index) to Bovada odds and suggests bets

### Organization
1. Get up to date FPI from ESPN- DONE
2. Get up to date odds from Bovada - difficult, use espn chalk instead
3. Money line implied odds comparison
4. Point spread implied odds comparison

_______________________________________________________________________________________________________________________________

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# up to date FPI function
# returns pandas dataframe with current FPI table

def getFPI():
    url = 'https://www.espn.com/nfl/fpi'
    r = requests.get(url)
    html = r.text
    
    # ESPN splits the FPI table into two sides
    
    # put the left table (team names) into a pandas dataframe    
    soup = BeautifulSoup(html)
    table1 = soup.find('table', {"class": "Table Table--align-right Table--fixed Table--fixed-left"})
    rows = table1.find_all('tr')
    teams_data = []
    for row in rows[2:]:
        cols = row.find_all('td')
        cols = [element.text.strip() for element in cols]
        teams_data.append([element for element in cols if element])   

    teams_df = pd.DataFrame(teams_data)
    
    # put the right side table (FPI and other stats) into a pandas dataframe
    table2 = soup.find('table', {"class": "Table Table--align-right"})
    rows = table2.find_all('tr')
    stats_data = []
    for row in rows[2:]:
        cols = row.find_all('td')
        cols = [element.text.strip() for element in cols]
        stats_data.append([element for element in cols if element])   

    stats_df = pd.DataFrame(stats_data)
    
    # combine into one dataframe and update headings
    df = pd.merge(teams_df, stats_df, left_index=True, right_index=True)
    headers = {'0_x': 'TEAM', 
               '0_y': 'W-L', 
               1 : 'FPI', 
               2: 'RK', 
               3: 'TRND', 
               4: 'OFF', 
               5: 'DEF', 
               6: 'ST', 
               7: 'SOS', 
               8: 'REM_SOS', 
               9: 'AVG_WP'}
    df = df.rename(index=str,columns=headers)
    return df

In [3]:
def getESPNlines():
    url = 'https://www.espn.com/nfl/lines'
    dfs = pd.read_html(url)
    
    return dfs

In [4]:
dfs_in = getESPNlines()

In [5]:
line_dfs = dfs_in[:]
for i, df in enumerate(dfs_in):

    line_dfs[i] = df

## How to calculate implied odds (%) based on American Money Line
 
### Negative American Odds (favorite)
implied probability = negative odds / (negative odds + 100) * 100

### Positive American Odds (underdog)
implied probability = 100 / (positive odds + 100) * 100

In [6]:
# add implied odds column to dfs and compare
lines_list = line_dfs[:]
for i, df in enumerate(line_dfs):
    # sort: underdogs, favorites    
    df = df.sort_values('LINE', ascending=False)
    
    # add total and points for each team row
    total, spread = df.iloc[0]['LINE'], df.iloc[1]['LINE']
    df['TOTAL'] = [total, total]
    df['SPREAD'] = [-spread, spread]
    
    # drop unnecessary columns
    df = df.drop(['REC (ATS)', 'OPEN', 'LINE'], axis=1)
  
    # sort: underdog, favorite
    df = df.sort_values(['ML'], ascending=False)
    
    # calculate implied odds from ML
    for index, row in df.iterrows():
        if row['ML'] > 0:
            underdog = (100*100/(row['ML']+100))
        else:
            favorite = 100*abs(row['ML'])/(abs(row['ML'])+100)
    df['Implied_Odds'] = [underdog,favorite]
    
    # calculate ML edge (FPI - Implied Odds)
    df['FPI'] = df['FPI'].str.strip(' %')
    df['FPI'] = df['FPI'].astype(float)
    
    df['ML_Edge'] = df['FPI'] - df['Implied_Odds']
    df['U/F?'] = ['underdog', 'favorite']
    
    df.columns.values[0] = 'TEAM'
    
    lines_list[i] = df

In [7]:
# convert list of games to dataframe, organize columns, index by team
lines = pd.concat(lines_list)
lines = lines[['TEAM', 'ML', 'SPREAD', 'TOTAL', 'FPI', 'Implied_Odds', 'ML_Edge', 'U/F?']]
team_lines = lines.set_index('TEAM')

points_FPI = getFPI()
points_FPI = points_FPI.set_index('TEAM')
points_FPI['FPI'] = points_FPI['FPI'].astype(float)
points_FPI

,W-L,FPI,RK,TRND,OFF,DEF,ST,SOS,REM_SOS,AVG_WP
TEAM,,,,,,,,,,
Kansas City Chiefs,6-1,10.2,1,--,9.0,1.3,-0.2,17,26,4
Tampa Bay Buccaneers,5-2,8.3,2,--,2.8,6.0,-0.5,14,13,7
Baltimore Ravens,5-1,6.7,3,--,2.0,4.2,0.4,24,31,1
San Francisco 49ers,4-3,6.2,4,2,3.3,2.6,0.3,29,6,6
Los Angeles Rams,5-2,5.1,5,--,2.9,2.3,-0.1,21,2,9
Seattle Seahawks,5-1,4.8,6,2,5.3,-0.6,0.0,20,8,5
Green Bay Packers,5-1,4.1,7,2,5.6,-1.0,-0.4,4,24,3
New Orleans Saints,4-2,3.8,8,--,5.1,-1.7,0.3,19,4,16
Pittsburgh Steelers,6-0,3.6,9,2,-0.2,3.9,-0.1,31,25,2


# NOTE

ESPN SOS (strength of schedule) and AVG_WP (average in-game win probability) appear to be inversely related (intuitive), but discrepencies could lead to betting edge. Room for more analysis

### Money Line Bets Sorted by Best Edge
only displays ML bets with a positive edge

In [8]:
# take positive edge, round values, and organize
pos_edge = team_lines[team_lines.ML_Edge > 0]
pos_edge = pos_edge[['ML', 'FPI', 'Implied_Odds', 'ML_Edge', 'U/F?']]
pos_edge = pos_edge.sort_values('ML_Edge', ascending=False)
pos_edge.round(decimals=3)

,ML,FPI,Implied_Odds,ML_Edge,U/F?
TEAM,,,,,
Denver Broncos,150,53.5,40.000,13.500,underdog
Minnesota Vikings,235,38.9,29.851,9.049,underdog
Cincinnati Bengals,250,35.9,28.571,7.329,underdog
San Francisco 49ers,130,49.1,43.478,5.622,underdog
Dallas Cowboys,425,24.4,19.048,5.352,underdog
New England Patriots,180,40.9,35.714,5.186,underdog
Miami Dolphins,155,43.5,39.216,4.284,underdog
Chicago Bears,180,38.5,35.714,2.786,underdog
New York Giants,500,17.0,16.667,0.333,underdog


### Point Spreads
ESPN FPI is defined as the expected point margin vs an average opponent
Assume an average opponent has an FPI of zero

FPI favorite expected point spread = Favorite FPI - Underdog FPI
FPI underdog expected point spread = -FPI favorite expected point spread

In [9]:
# loop over each matchup and calculate
spread_list = lines_list.copy()
for i, df in enumerate(lines_list):
    
    # drop unnecessary columns
    df = df.drop(['ML', 'FPI', 'TOTAL', 'Implied_Odds', 'ML_Edge'], axis=1)

    # index to teams
    df = df.set_index('TEAM')

    # add FPI, calculate FPI spread, calculate points edge
    df = pd.concat([df, points_FPI['FPI']], axis=1, join='inner')
    fpi_spread = df.iloc[0]['FPI'] - df.iloc[1]['FPI']
    df['FPI_spread'] = [-fpi_spread, fpi_spread]
    df['Points_Edge'] = df['SPREAD'] - df['FPI_spread']
     
    spread_list[i] = df

In [10]:
# convert list to a dataframe, organize, take positive edge
spreads = pd.concat(spread_list)
spreads = spreads[['SPREAD', 'FPI', 'FPI_spread', 'Points_Edge', 'U/F?']]

points_edge = spreads[spreads.Points_Edge > 0]
points_edge = points_edge.sort_values('Points_Edge', ascending=False)
points_edge

,SPREAD,FPI,FPI_spread,Points_Edge,U/F?
TEAM,,,,,
San Francisco 49ers,3.0,6.2,-1.4,4.4,underdog
Dallas Cowboys,11.0,-8.3,7.0,4.0,underdog
Las Vegas Raiders,2.5,-0.7,-0.5,3.0,underdog
Los Angeles Rams,-3.5,5.1,-6.3,2.8,favorite
Denver Broncos,3.0,-3.0,0.4,2.6,underdog
Tampa Bay Buccaneers,-12.5,8.3,-14.9,2.4,favorite
New England Patriots,4.0,-0.5,1.7,2.3,underdog
Minnesota Vikings,6.0,-0.1,4.2,1.8,underdog
New York Jets,20.5,-8.8,19.0,1.5,underdog


__________________________________________________________

# Final Results

## Money Line Edge

In [11]:
pos_edge

,ML,FPI,Implied_Odds,ML_Edge,U/F?
TEAM,,,,,
Denver Broncos,150,53.5,40.000000,13.500000,underdog
Minnesota Vikings,235,38.9,29.850746,9.049254,underdog
Cincinnati Bengals,250,35.9,28.571429,7.328571,underdog
San Francisco 49ers,130,49.1,43.478261,5.621739,underdog
Dallas Cowboys,425,24.4,19.047619,5.352381,underdog
New England Patriots,180,40.9,35.714286,5.185714,underdog
Miami Dolphins,155,43.5,39.215686,4.284314,underdog
Chicago Bears,180,38.5,35.714286,2.785714,underdog
New York Giants,500,17.0,16.666667,0.333333,underdog


## Points Edge

In [12]:
points_edge

,SPREAD,FPI,FPI_spread,Points_Edge,U/F?
TEAM,,,,,
San Francisco 49ers,3.0,6.2,-1.4,4.4,underdog
Dallas Cowboys,11.0,-8.3,7.0,4.0,underdog
Las Vegas Raiders,2.5,-0.7,-0.5,3.0,underdog
Los Angeles Rams,-3.5,5.1,-6.3,2.8,favorite
Denver Broncos,3.0,-3.0,0.4,2.6,underdog
Tampa Bay Buccaneers,-12.5,8.3,-14.9,2.4,favorite
New England Patriots,4.0,-0.5,1.7,2.3,underdog
Minnesota Vikings,6.0,-0.1,4.2,1.8,underdog
New York Jets,20.5,-8.8,19.0,1.5,underdog


___________________________________________________________

# Stuff to Add

* add opponent, home/away, record, weather to dataframes
* add strength of schedule to date anaylsis